In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizerFast
import torch
import sys
import os
from peft import PeftModel, PeftConfig
from peft import LoraConfig, TaskType, get_peft_model

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from transformers import LlamaForCausalLM, LlamaTokenizer


sys.path.append('../../JS')
from honest_llama import llama


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home//venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home//venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home//venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//xen03.iitd.ac.in'), PosixPath('3128')}
  warn(msg)
/home//venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home//venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('module'), PosixPath('//matplotlib_inline.backend_inline')}
  warn(msg)
/home//venv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib64')}
  warn(msg)
/home/

In [2]:
path_to_model = '/home/models//Llama-2-7b-hf'  # push to runtime argument
path_to_tokenizer = '/home/models//Llama-2-7b-hf' # push to runtime argument
# ../frugal_lms/saved_model/base_gptj/
# %%
device = 'cuda:0' # push it to runtime argument
# device = args.device
# mt = ModelAndTokenizer(path_to_model, device=device)
# tokenizer = LlamaTokenizerFast.from_pretrained('hf-internal-testing/llama-tokenizer', is_fast=True)
# model = AutoModelForCausalLM.from_pretrained(path_to_model, low_cpu_mem_usage=True).to(device)

tokenizer = LlamaTokenizer.from_pretrained(path_to_tokenizer)
model = llama.LLaMAForCausalLM.from_pretrained(path_to_model, low_cpu_mem_usage=True).to(device)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home//venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home//venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [3]:
import json

In [4]:
import json
with open('data/2shot10k.json', 'r') as f:
    data = json.load(f)

In [10]:
print(data[0])

{'question': 'Rompuses are sterpuses. Rompuses are zumpuses. Rompuses are not medium. Grimpuses are rompuses. Impuses are medium. Grimpuses are tumpuses. Grimpuses are shy. Each zumpus is temperate. Each tumpus is sunny. Each numpus is a lorpus. Numpuses are liquid. Sally is a grimpus. Sally is a numpus.', 'query': 'True or false: Sally is not medium.', 'answer': 'Sally is a grimpus. Grimpuses are rompuses. Sally is a rompus. Rompuses are not medium. Sally is not medium. True', 'label': 'True', 'tree': '(sterpus\n  (rompus properties: not medium\n    (grimpus properties: shy\n    )\n  )\n)\n(zumpus properties: temperate\n  (rompus properties: not medium\n    (grimpus properties: shy\n    )\n  )\n)\n(tumpus properties: sunny\n  (grimpus properties: shy\n  )\n)\n(lorpus\n  (numpus properties: liquid\n  )\n)\n'}


In [12]:
import pickle
with open('data/pluralToNouns.pickle', 'rb') as handle:
    pluralToNouns = pickle.load(handle)
def preprocessing(word):
    word = word.lower()
    # print(word)
    if(word in pluralToNouns):
        word = pluralToNouns[word]
    return word

In [13]:
def tokenize_sentence_with_parent(sentence, tokenizer):
    tokenized_sentence = tokenizer.tokenize(sentence)
    # print(tokenized_sentence)
    # print(tokenized_sentence)
    parent_words = []

    i = 0
    current_word = ""

    for i in range(len(tokenized_sentence)):
        if(tokenized_sentence[i] == '.'):
            parent_words.append('.')
        # print(tokenized_sentence[i].startswith("▁"))
        elif tokenized_sentence[i].startswith("▁") and (i + 1 < len(tokenized_sentence)) and tokenized_sentence[i + 1].startswith("▁"):
            parent_words.append(tokenized_sentence[i].replace("▁", ""))
        elif(tokenized_sentence[i].startswith("▁") and (i  == (len(tokenized_sentence) - 1))):
            parent_words.append(tokenized_sentence[i].replace("▁", ""))
        elif(tokenized_sentence[i].startswith("▁") == True and (i + 1 < len(tokenized_sentence)) and tokenized_sentence[i + 1].startswith("▁") == False):
            end = i +  1
            while(end < len(tokenized_sentence)):
                if(tokenized_sentence[end].startswith("▁") == True or tokenized_sentence[end] == '.'):
                    end -=  1
                    break
                end += 1
            start = i
            notPresent = False
            while(start > 0):
                if(tokenized_sentence[start] == '.'):
                    start = i
                    break
                if(tokenized_sentence[start] == '▁not'):
                    notPresent = True
                    break
                start -= 1
            if(notPresent == False):
                start = i

            parent_word = ""
            # print(end)
            for j in range(start, end + 1):
                if(j < len(tokenized_sentence) - 1):
                    parent_word += tokenizer.convert_tokens_to_string([tokenized_sentence[j]])
            # print("parent word: ", parent_word)
            # print(start, end)
            parent_words.append(parent_word)

             
        elif(tokenized_sentence[i].startswith("▁") == False):
            temp = []
            start = i
            # print(tokenized_sentence[i])
            while(start > 0):
                if(tokenized_sentence[start].startswith("▁") == True or tokenized_sentence[start] == '.'):
                    break
                start -= 1
            
            start2 = i
            notPresent = False
            while(start2 > 0):
                if(tokenized_sentence[start2] == '.'):
                    start2 = i
                    break
                if(tokenized_sentence[start2] == '▁not'):
                    notPresent = True
                    break
                start2 -= 1
            if(notPresent == True):
                start = start2
            # print("start: ",tokenized_sentence[start])
            end = i
            while(end < len(tokenized_sentence)):
                if(tokenized_sentence[end].startswith("▁") == True or tokenized_sentence[end] == '.'):
                    end -=  1
                    break
                end += 1
            # print("end: ",tokenized_sentence[end])
            parent_word = ""
            for j in range(start, end + 1):
                if(j < len(tokenized_sentence) - 1):
                # print(tokenized_sentence[j])
                    parent_word += tokenizer.convert_tokens_to_string([tokenized_sentence[j]])
            # print("parent word: ", parent_word)
            parent_words.append(parent_word)
    for i in range(len(parent_words)):
        parent_words[i] = preprocessing(parent_words[i])
    # print(tokenized_sentence)
    # print(parent_words)
    return tokenized_sentence, parent_words

In [15]:
tokenized_sentence, parent_words = tokenize_sentence_with_parent(data[0]['question'], tokenizer)

In [18]:
for i in range(len(parent_words)):
    print(tokenized_sentence[i], parent_words[i])

▁Rom rompus
p rompus
uses rompus
▁are are
▁ster sterpus
p sterpus
uses sterpus
. .
▁Rom rompus
p rompus
uses rompus
▁are are
▁zum zumpus
p zumpus
uses zumpus
. .
▁Rom rompus
p rompus
uses rompus
▁are are
▁not not
▁medium notmedium
. .
▁Gr grimpus
imp grimpus
uses grimpus
▁are are
▁rom rompus
p rompus
uses rompus
. .
▁Imp impus
uses impus
▁are are
▁medium medium
. .
▁Gr grimpus
imp grimpus
uses grimpus
▁are are
▁t tumpus
ump tumpus
uses tumpus
. .
▁Gr grimpus
imp grimpus
uses grimpus
▁are are
▁sh shy
y shy
. .
▁Each each
▁zum zumpus
pus zumpus
▁is is
▁temper temperate
ate temperate
. .
▁Each each
▁t tumpus
ump tumpus
us tumpus
▁is is
▁sun sunny
ny sunny
. .
▁Each each
▁num numpus
pus numpus
▁is is
▁a a
▁l lorpus
or lorpus
pus lorpus
. .
▁N numpus
ump numpus
uses numpus
▁are are
▁liquid liquid
. .
▁S sally
ally sally
▁is is
▁a a
▁gr grimpus
imp grimpus
us grimpus
. .
▁S sally
ally sally
▁is is
▁a a
▁num numpus
pus numpus
. .


In [21]:
class TreeNode:
    def __init__(self, name, children=None):
        self.name = name
        self.parentName = None
        self.children = children if children else []

def parseTreeString(tree_structure):

    current_node = TreeNode('root')
    topNode = current_node
    # print(topNode.name)
    def extractPropertyAndNode(line):
        property_name = None
        if('properties:' in line):
            property_name = line.split(':')[-1].strip().replace(' ', '')
            node_name = line.split('properties:')[0].strip().replace('(', '').replace(')', '')
        else:
            node_name = line.strip().replace('(', '').replace(')', '')
        return node_name, property_name
    # Split the tree structure into lines
    lines = tree_structure.split("\n")
    # print(len(lines))
    for line in lines:
        line = line.strip("()")

        node_name, property_name = extractPropertyAndNode(line)
        # print(node_name, property_name)
        if(node_name == ''):
            continue
        if(property_name is None):
            property_name = ''

        name_node = TreeNode(node_name)
        property_node = TreeNode(property_name)

        if(current_node is None):
            current_node = name_node
            if(property_name != ''):
                current_node.children.append(property_node)
        else:
            current_node.children.append(name_node)
            current_node = name_node
            if(property_name != ''):
                current_node.children.append(property_node)
                
    topNode = topNode.children[0]

    return topNode

In [22]:
entry = 0
tree_structure = data[entry]['tree']

In [23]:
structures = tree_structure.split(')\n(')

In [24]:
root_node = parseTreeString(structures[0])

In [31]:
print(structures[0])

(sterpus
  (rompus properties: not medium
    (grimpus properties: shy
    )
  )



In [34]:
root_node.children[0].children[0].name

'notmedium'

In [35]:
def get_nodes(root):
    nodes = []
    stack = [root]
    while stack:
        node = stack.pop()
        nodes.append(node)
        stack.extend(node.children)
    return nodes


def fill_adjacency_matrix(node, adjacency_matrix, node_indices):
    node_index = node_indices[node]
    for child in node.children:
        child_index = node_indices[child]
        adjacency_matrix[node_index][child_index] = 1
        fill_adjacency_matrix(child, adjacency_matrix, node_indices)

def create_adjacency_matrix(root):
    nodes = get_nodes(root)
    num_nodes = len(nodes)
    adjacency_matrix = [[0] * num_nodes for _ in range(num_nodes)]
    node_indices = {node: index for index, node in enumerate(nodes)}
    fill_adjacency_matrix(root, adjacency_matrix, node_indices)
    return adjacency_matrix

In [36]:
adjacency_matrix = create_adjacency_matrix(root_node)

In [38]:
nodes = get_nodes(root_node)

In [41]:
node_name = []
for node in nodes:
    print(node.name)
    node_name.append(node.name)

sterpus
rompus
grimpus
shy
notmedium


In [42]:
node_name

['sterpus', 'rompus', 'grimpus', 'shy', 'notmedium']

In [43]:
adjacency_matrix

[[0, 1, 0, 0, 0],
 [0, 0, 1, 0, 1],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0]]

In [88]:
print(tree_structure)

(sterpus
  (rompus properties: not medium
    (grimpus properties: shy
    )
  )
)
(zumpus properties: temperate
  (rompus properties: not medium
    (grimpus properties: shy
    )
  )
)
(tumpus properties: sunny
  (grimpus properties: shy
  )
)
(lorpus
  (numpus properties: liquid
  )
)



In [44]:
root_nodes = []
adjacency_matrixs = []
allNodes = []
structures = tree_structure.split(')\n(')
for structure in structures:
    root_node = parseTreeString(structure)
    adjacency_matrix = create_adjacency_matrix(root_node)
    allNode = get_nodes(root_node)
    root_nodes.append(root_node)
    adjacency_matrixs.append(adjacency_matrix)
    allNodes.append(allNode)

In [46]:
tokenized_sentence, parent_words = tokenize_sentence_with_parent(data[entry]['question'], tokenizer)

In [47]:
def findIndexInMatrix(nodes, word, parent_word):
    for i in range(len(nodes)):
        if(nodes[i].name == parent_word):
            return i
    return None

In [72]:
n = len(tokenized_sentence)
matrix = [[0 for i in range(n)] for j in range(n)]
for i in range(len(tokenized_sentence)):
    for j in range(i,len(tokenized_sentence)):
        if(i == j):
            matrix[i][j] = 0
        for count in range(len(root_nodes)):
            nodes = allNodes[count]
            firstIndex = findIndexInMatrix(nodes, tokenized_sentence[i], parent_words[i])
            secondIndex = findIndexInMatrix(nodes, tokenized_sentence[j], parent_words[j])
            if(firstIndex != None and secondIndex != None):
                if(adjacency_matrixs[count][firstIndex][secondIndex] == 1 or adjacency_matrixs[count][secondIndex][firstIndex] == 1):
                    # print(parent_words[i], parent_words[j])
                    matrix[i][j] = 1
                    matrix[j][i] = 1

In [52]:
allNodeEntities = []
for nodes in allNodes:
    for node in nodes:
        allNodeEntities.append(node.name)

In [53]:
allNodeEntities

['sterpus',
 'rompus',
 'grimpus',
 'shy',
 'notmedium',
 'zumpus',
 'rompus',
 'grimpus',
 'shy',
 'notmedium',
 'temperate',
 'tumpus',
 'grimpus',
 'shy',
 'sunny',
 'lorpus',
 'numpus',
 'liquid']

In [56]:
len(matrix)

96

In [60]:
def getPositiveAndNegativePair(tokenized_sentence, parent_words, matrix, typeA, typeB):
    positivePair = []
    negativePair = []
    start = 0
    end = len(tokenized_sentence)
    for i in range(start, end):
        if(typeA == 'first'):
            if(i >= 0 and parent_words[i - 1] == parent_words[i]):
                continue
        elif(typeA == 'last'):
            if(i < (len(tokenized_sentence) - 1) and parent_words[i] == parent_words[i + 1]):
                continue
        j = i
        while(j < len(tokenized_sentence)):
            if(tokenized_sentence[j] == '.'):
                break
            if(typeB == 'last'):
                if(matrix[i][j] == 1 and j < (len(tokenized_sentence) - 1) and matrix[i][j + 1] != 1):
                    if('not' in parent_words[j]):
                        # print(tokenized_sentence[i], tokenized_sentence[j], parent_words[i], parent_words[j])
                        negativePair.append((i, j))
                    else:
                        # print(tokenized_sentence[i], tokenized_sentence[j], parent_words[i], parent_words[j])
                        positivePair.append((i, j))
                    break
            else:
                if(matrix[i][j] == 1):
                    if('not' in parent_words[j]):
                        # print(tokenized_sentence[i], tokenized_sentence[j], parent_words[i], parent_words[j])
                        negativePair.append((i, j))
                    else:
                        # print(tokenized_sentence[i], tokenized_sentence[j], parent_words[i], parent_words[j])
                        positivePair.append((i, j))
                    if(typeB == 'first'):
                        break
            j += 1
    return positivePair, negativePair

In [66]:
print(parent_words)
print(tokenized_sentence)

['rompus', 'rompus', 'rompus', 'are', 'sterpus', 'sterpus', 'sterpus', '.', 'rompus', 'rompus', 'rompus', 'are', 'zumpus', 'zumpus', 'zumpus', '.', 'rompus', 'rompus', 'rompus', 'are', 'not', 'notmedium', '.', 'grimpus', 'grimpus', 'grimpus', 'are', 'rompus', 'rompus', 'rompus', '.', 'impus', 'impus', 'are', 'medium', '.', 'grimpus', 'grimpus', 'grimpus', 'are', 'tumpus', 'tumpus', 'tumpus', '.', 'grimpus', 'grimpus', 'grimpus', 'are', 'shy', 'shy', '.', 'each', 'zumpus', 'zumpus', 'is', 'temperate', 'temperate', '.', 'each', 'tumpus', 'tumpus', 'tumpus', 'is', 'sunny', 'sunny', '.', 'each', 'numpus', 'numpus', 'is', 'a', 'lorpus', 'lorpus', 'lorpus', '.', 'numpus', 'numpus', 'numpus', 'are', 'liquid', '.', 'sally', 'sally', 'is', 'a', 'grimpus', 'grimpus', 'grimpus', '.', 'sally', 'sally', 'is', 'a', 'numpus', 'numpus', '.']
['▁Rom', 'p', 'uses', '▁are', '▁ster', 'p', 'uses', '.', '▁Rom', 'p', 'uses', '▁are', '▁zum', 'p', 'uses', '.', '▁Rom', 'p', 'uses', '▁are', '▁not', '▁medium', '.

In [62]:
positivePair, negativePair = getPositiveAndNegativePair(tokenized_sentence, parent_words, matrix, 'first', 'first')


In [69]:
data[entry]['question']

'Rompuses are sterpuses. Rompuses are zumpuses. Rompuses are not medium. Grimpuses are rompuses. Impuses are medium. Grimpuses are tumpuses. Grimpuses are shy. Each zumpus is temperate. Each tumpus is sunny. Each numpus is a lorpus. Numpuses are liquid. Sally is a grimpus. Sally is a numpus.'

In [85]:
tokenized_sentence[34]

'▁medium'

In [86]:
matrix[31][34]

0

In [74]:
for pair in positivePair:
    print(pair)
    print(parent_words[pair[0]], parent_words[pair[1]])

(0, 4)
rompus sterpus
(8, 12)
rompus zumpus
(23, 27)
grimpus rompus
(36, 40)
grimpus tumpus
(44, 48)
grimpus shy
(52, 55)
zumpus temperate
(59, 63)
tumpus sunny
(67, 71)
numpus lorpus
(75, 79)
numpus liquid


In [70]:
for pair in negativePair:
    print(parent_words[pair[0]], parent_words[pair[1]])

rompus notmedium


In [73]:
def getUnRelatedPair(tokenized_sentence, parent_words, adjacency_matrix, allNodeEntities, typeA, typeB):
    unrelatedPair = []
    start = 0
    end = len(tokenized_sentence)
    if(typeA == 'first'):
        start = 1
    if(typeA == 'last'):
        end = len(tokenized_sentence) - 1
    for i in range(start, end):
        if(typeA == 'first'):
            if(parent_words[i - 1] == parent_words[i]):
                continue
        elif(typeA == 'last'):
            if(parent_words[i] == parent_words[i + 1]):
                continue
        for j in range(i, len(tokenized_sentence)):
            if(typeB == 'last'):
                if(parent_words[i] != parent_words[j] and parent_words[i] in allNodeEntities and parent_words[j] in allNodeEntities and adjacency_matrix[i][j] == float('inf')
                   and j < (len(tokenized_sentence) - 1) and parent_words[j] != parent_words[j + 1]):
                    # print(tokenized_sentence[i], tokenized_sentence[j], parent_words[i], parent_words[j])
                    unrelatedPair.append((i, j))
                    break

            else:
                if(parent_words[i] != parent_words[j] and parent_words[i] in allNodeEntities and parent_words[j] in allNodeEntities and adjacency_matrix[i][j] == float('inf')):
                    # print(tokenized_sentence[i], tokenized_sentence[j], parent_words[i], parent_words[j])
                    unrelatedPair.append((i, j))
                    if(typeB == 'first'):
                        break
    return unrelatedPair

In [89]:
import numpy as np
def adjacency_to_distance(adj_matrix):
    num_vertices = len(adj_matrix)
    distance_matrix = np.full((num_vertices, num_vertices), float('inf'))

    for i in range(num_vertices):
        distance_matrix[i][i] = 0

    for i in range(num_vertices):
        for j in range(num_vertices):
            if adj_matrix[i][j] != 0:
                distance_matrix[i][j] = adj_matrix[i][j]

    for k in range(num_vertices):
        distance_matrix = np.minimum(distance_matrix, distance_matrix[:, k:k+1] + distance_matrix[k:k+1, :])

    return distance_matrix.tolist()

In [90]:
distanceMatrix = adjacency_to_distance(matrix)

In [91]:
unrelatedPair = getUnRelatedPair(tokenized_sentence, parent_words, distanceMatrix, allNodeEntities, 'first', 'first')


In [93]:
for pair in unrelatedPair:
    print(parent_words[pair[0]], parent_words[pair[1]])

sterpus numpus
rompus numpus
zumpus numpus
rompus numpus
notmedium numpus
grimpus numpus
rompus numpus
grimpus numpus
tumpus numpus
grimpus numpus
shy numpus
zumpus numpus
temperate numpus
tumpus numpus
sunny numpus
numpus grimpus
lorpus grimpus
numpus grimpus
liquid grimpus
grimpus numpus


In [95]:
unrelatedPair

[(4, 67),
 (8, 67),
 (12, 67),
 (16, 67),
 (21, 67),
 (23, 67),
 (27, 67),
 (36, 67),
 (40, 67),
 (44, 67),
 (48, 67),
 (52, 67),
 (55, 67),
 (59, 67),
 (63, 67),
 (67, 85),
 (71, 85),
 (75, 85),
 (79, 85),
 (85, 93)]

In [98]:
question = data[entry]['question'] + ' ' + data[entry]['query']
tree_structure = data[entry]['tree']
nShots = f"{question} Let us think step by step."

In [99]:
nShots

'Rompuses are sterpuses. Rompuses are zumpuses. Rompuses are not medium. Grimpuses are rompuses. Impuses are medium. Grimpuses are tumpuses. Grimpuses are shy. Each zumpus is temperate. Each tumpus is sunny. Each numpus is a lorpus. Numpuses are liquid. Sally is a grimpus. Sally is a numpus. True or false: Sally is not medium. Let us think step by step.'

In [100]:
input_ids = tokenizer(nShots, return_tensors="pt").input_ids.to(device)

In [102]:
outputs = model.generate(input_ids = input_ids, output_hidden_states=True, return_dict_in_generate=True, max_new_tokens=1)


/home//venv/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


In [103]:
text = tokenizer.decode(outputs.sequences[0][len(input_ids[0]):].tolist(), clean_up_tokenization_spaces=True).strip()

In [104]:
print(text)

Let


In [108]:
len(outputs.hidden_states[0])

33